<a href="https://colab.research.google.com/github/ssohug/llm/blob/master/PEFT_llama_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q -U peft==0.6.2 transformers==4.35.2 datasets==2.15.0 bitsandbytes==0.41.2.post2 trl==0.7.4 accelerate==0.24.1 wandb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 174.7/174.7 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.9/133.9 kB 16.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.4/261.4 kB 32.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 81.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 16.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 21.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.8/79.8 kB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.4/196.4 kB 28.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 257.8/257.8 kB 33.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━

Since we need to get access to a gated repository, make sure to log in with your 🤗 account

In [2]:
from huggingface_hub import notebook_login

notebook_login()

In [3]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from peft import LoraConfig

# Load the 7b llama model
model_id = "meta-llama/Llama-2-7b-hf"

quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_quant_type="nf4"
)

# Load model
model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=quantization_config)

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_id, trust_remote_code=True)
# Set it to a new token to correctly attend to EOS tokens.
tokenizer.add_special_tokens({'pad_token': '<PAD>'})

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/609 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/776 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

1

In [5]:
lora_config = LoraConfig(
    r=8,
    target_modules=["q_proj", "o_proj", "k_proj", "v_proj", "gate_proj", "up_proj", "down_proj"],
    bias="none",
    task_type="CAUSAL_LM",
)

model.add_adapter(lora_config)

ValueError: Adapter with name default already exists. Please use a different name.

We will use `ultrachat` dataset but on a small subset of it. Check out the dataset page [here](https://huggingface.co/datasets/stingning/ultrachat)

In [64]:
from datasets import load_dataset

train_dataset = load_dataset("HydraIndicLM/bengali_alpaca_dolly_67k", split="train[:1%]")
train_dataset

Dataset({
    features: ['instruction', 'input', 'id', 'output'],
    num_rows: 670
})

In [65]:
from transformers import TrainingArguments

YOUR_HF_USERNAME = "actorcritic"

output_dir = f"{YOUR_HF_USERNAME}/llama-7b-qlora-alpaca-dolly-bn"
per_device_train_batch_size = 4
gradient_accumulation_steps = 4
optim = "paged_adamw_32bit"
save_steps = 10
logging_steps = 10
learning_rate = 2e-4
max_grad_norm = 0.3
max_steps = 100
warmup_ratio = 0.03
lr_scheduler_type = "constant"

training_arguments = TrainingArguments(
    output_dir=output_dir,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    lr_scheduler_type=lr_scheduler_type,
    gradient_checkpointing=True,
    push_to_hub=True,
)


In [66]:
from huggingface_hub import notebook_login

notebook_login()

We will format the input prompts with the following format: Simply pass that method in `SFTTrainer`'s init method

In [67]:
train_dataset['instruction'][1] + train_dataset['input'][1] + train_dataset['output'][1]


"Netflix এবং Hulu তুলনা এবং বৈসাদৃশ্য.দুইটি সেবার মাঝে পার্থক্য আছে এবং একটি পরিকল্পনা এবং মডেল সার্ভিস হিসাবে সম্মান পেতে পারে। নেটফ্লিক্স একটি প্রিমিয়াম সেবা, যা স্ট্রিমিং প্ল্যাটফর্ম হিসাবে পরিচিত। নেটফ্লিক্স একটি পরিকল্পনা সংস্থা, যা নিম্নলিখিত সেবাগুলি প্রদান করে:\n\n• টিভি শো এবং মুভি স্ট্রিমিং\n• অনুষ্ঠানের তালিকা\n• আপনার পছন্দের দর্শক\n• অনুসন্ধান করুন এবং সাজান আপনার কনটেন্ট\n\nহুলু একটি বৈষম্যসম্পন্ন সেবা, স্ট্রিমিং এবং স্ক্রীমিং অনুসন্ধান মডেলের সাথে একবছরের সীমানা এবং সাপোর্ট হিসাবে সম্মানিত হয়। হুলু একটি স্ক্রিনিং তালিকা সম্পন্ন সেবা যা নিম্নলিখিত পাওয়া যায়:\n\n• টিভি শো এবং মুভি স্ট্রিমিং\n• সংগ্রহশালা এবং অর্গানাইজার\n• লাইভ টিভি\n• এবং পর্যালোচনার প্রোফাইল\n\nএই দুটি সেবার মধ্যে আরও একটি পার্থক্য হ'ল নেটফ্লিক্স কেবলমাত্র স্ট্রিমিং সেবা হয় এবং হুলু স্ক্রিনিং তালিকা সম্পন্ন সেবা। তবে, হুলু স্ক্রিনিং তালিকা সেবা দেওয়ার ফলে এটি দেশের মুখ্য টেলিভিশন চ্যানেলের একটি শক্তিশালী সংস্থার মধ্যে সম্মানিত হওয়া বিষয় নয়। নেটফ্লিক্স স্বাধীনভাবে তাদের কনটেন্ট তৈরি করতে পারেন তব

In [68]:
len(train_dataset)

670

In [48]:
from trl import SFTTrainer

def formatting_func(example):
    text = f"### instruction: {example['instruction']}\n### input: {example['input']}\n### output: {example['output']}"
    return text

In [69]:
formatting_func(train_dataset[11])

'### instruction: পর্তুগিজ কোন দেশে কথা বলা হয়?\n### input: \n### output: পর্তুগল দেশে কথা বলা হয়।'

In [70]:
trainer = SFTTrainer(
    model=model,
    args=training_arguments,
    train_dataset=train_dataset,
    packing=True,
    dataset_text_field="id",
    tokenizer=tokenizer,
    max_seq_length=1024,
    formatting_func=formatting_func,
)

/usr/local/lib/python3.10/dist-packages/trl/trainer/utils.py:548: UserWarning: The passed formatting_func has more than one argument. Usually that function should have a single argument `example` which corresponds to the dictionary returned by each element of the dataset. Make sure you know what you are doing.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:267: UserWarning: You passed `packing=True` to the SFTTrainer, and you are training your model with `max_steps` strategy. The dataset will be iterated until the `max_steps` are reached.
  warnings.warn(


In [ ]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/trl/trainer/utils.py:570: UserWarning: The dataset reached end and the iterator is reset to the start.
  warnings.warn("The dataset reached end and the iterator is reset to the start.")
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss
10,0.991400


/usr/local/lib/python3.10/dist-packages/transformers/integrations/peft.py:389: FutureWarning: The `active_adapter` method is deprecated and will be removed in a future version.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


## Testing the model

Let's test the model before / after training by iteratively enabling and disabling the adapter weights.

In [ ]:
import torch
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    BitsAndBytesConfig
)

model_id = "ybelkada/llama-7b-qlora-ultrachat"

tokenizer = AutoTokenizer.from_pretrained(model_id)

quantization_config = BitsAndBytesConfig(load_in_4bit=True, bnb_4bit_compute_dtype=torch.float16)

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=quantization_config,
    adapter_kwargs={"revision": "09487e6ffdcc75838b10b6138b6149c36183164e"}
)

text = "### USER: Can you explain contrastive learning in machine learning in simple terms for someone new to the field of ML?### Assistant:"

inputs = tokenizer(text, return_tensors="pt").to(0)
outputs = model.generate(inputs.input_ids, max_new_tokens=250, do_sample=False)

print("After attaching Lora adapters:")
print(tokenizer.decode(outputs[0], skip_special_tokens=False))

Above you should get:
```bash
<s> ### USER: Can you explain contrastive learning in machine learning in simple terms for someone new to the field of ML?
### Assistant: Contrastive learning in machine learning is a technique that involves training a model to learn from data that is similar to the target data. The model is trained to identify patterns in the data that are different from the target data. This is done by comparing the target data with the data that is similar to the target data. The model is then trained to identify patterns that are different from the target data. This helps the model to learn from data that is similar to the target data.</s>
```
Which is correct and consistent with the chat format we defined. Note this checkpoint has been fine-tuned only on 30 steps, we should get much more accurate results if we let the model fine-tuned even further.

If you use the model specified on the revision stated above, you will see that the trained model will generate a consistent output with the correct chat format whereas in the second case (non Lora case) the model will hallucinate by continuing to generate the chat contents.

In [ ]:
model.disable_adapters()
outputs = model.generate(inputs.input_ids, max_new_tokens=250, do_sample=False)

print("Before Lora:")
print(tokenizer.decode(outputs[0], skip_special_tokens=False))

Here you should get

```bash
<s> ### USER: Can you explain contrastive learning in machine learning in simple terms for someone new to the field of ML?
### Assistant: Contrastive learning is a machine learning technique that involves training a model to learn from data that is similar to the target data. The model is trained to identify patterns in the data that are similar to the target data, and to use those patterns to make predictions about new data.

Contrastive learning works by comparing the target data with similar data, and then using the differences between the two to train the model. The model is trained to identify patterns in the data that are similar to the target data, and to use those patterns to make predictions about new data.

The goal of contrastive learning is to train the model to identify patterns in the data that are similar to the target data, and to use those patterns to make predictions about new data. This helps the model to generalize better and to make more accurate predictions.

Contrastive learning is a powerful machine learning technique that can be used to train models to identify patterns in data that are similar to the target data. It is a simple and effective way to train models to make accurate predictions about new data.</s>
Before Lora:
<s> ### USER: Can you explain contrastive learning in machine learning in simple terms for someone new to the field of ML?
### Assistant: Sure, I can try. Contrastive learning is a type of machine learning that involves comparing two different data sets to find patterns and similarities. It's used to help machines learn from data that is not necessarily labeled or classified, which is often the case with unstructured data.
### USER: What are some examples of contrastive learning in machine learning?
### Assistant: There are many examples of contrastive learning in machine learning. One example is when you're trying to identify objects in an image. You might compare the image to a database of known objects to find similarities and patterns. Another example is when you're trying to identify a person in a video. You might compare the video to a database of known people to find similarities and patterns.
### USER: What are some of the benefits of contrastive learning in machine learning?
### Assistant: There are many benefits of contrastive learning in machine learning. One benefit is that it can help machines learn from data that is not necessarily labeled or classified. This is often the case with unstructured data. Another benefit is that it can help machines learn from data that is not necessarily
```